In [81]:
import math
from collections import Counter
from matplotlib import pyplot as plt
import random
import pandas as pd
import os
import sys
import csv
import dateutil.parser
from packages.Statistics import correlation
from collections import Counter
import numpy as np

In this module, we are going to explore the trend of an each individual car brand and its price drop with the respect to the mileage and manufactured year. First, I have to collect data from outside source. I am using '1.2 Million Used Car Listings' provided by Evan Payne from Kaggle.

In [82]:
raw_data = []
raw_data = pd.read_csv("datasets/true_car_listings.csv")
# Credit: https://www.kaggle.com/jpayne/852k-used-car-listings/version/2#true_car_listings.csv

In [83]:
raw_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


In [84]:
print('Total length of the data : %d' %len(raw_data['Make']))

Total length of the data : 852122


In [85]:
print('Total number of the car brands : %d' %len(Counter(raw_data['Make'])))

Total number of the car brands : 58


In [86]:
Counter(raw_data['Make'])

Counter({'Acura': 11049,
         'Alfa': 44,
         'AM': 19,
         'Aston': 149,
         'Audi': 12618,
         'Bentley': 367,
         'BMW': 32415,
         'Buick': 12491,
         'Cadillac': 15047,
         'Chevrolet': 102268,
         'Chrysler': 16357,
         'Dodge': 34368,
         'Ferrari': 345,
         'FIAT': 1782,
         'Fisker': 19,
         'Ford': 110432,
         'Freightliner': 11,
         'Genesis': 141,
         'Geo': 2,
         'GMC': 29008,
         'Honda': 50193,
         'HUMMER': 949,
         'Hyundai': 35837,
         'INFINITI': 12258,
         'Isuzu': 76,
         'Jaguar': 2200,
         'Jeep': 40373,
         'Kia': 28636,
         'Lamborghini': 121,
         'Land': 4096,
         'Lexus': 20641,
         'Lincoln': 7120,
         'Lotus': 42,
         'Maserati': 1047,
         'Maybach': 10,
         'Mazda': 13365,
         'McLaren': 47,
         'Mercedes-Benz': 26323,
         'Mercury': 1076,
         'MINI': 4375,
       

There are about 80,000 individual data values about used cars, and 58 different car brands in the dataset. In order to make this data more general and useful, popular car brands are selected. According to 2018 US Automotive Brand Rankings – YTD, we are going to use only top 18 car brands according to their market share in the car industry.

credit: http://www.goodcarbadcar.net/2018/10/u-s-auto-sales-brand-rankings-september-2018-ytd/

In [87]:
top_brands = pd.read_csv("datasets/us_top_18_brands_2018.csv")

In [88]:
top_brands

,Brand,Month,Change,YTD,YTD Change,Month Share,YTD Share
0,Ford,"188,328",-11.52,"1,808,326",-1.81,13.07,13.87
1,Toyota,"178,501",-10.94,"1,706,410",5.88,12.39,13.09
2,Chevrolet,"159,171",-20.34,"1,486,515",-1.96,11.05,11.40
3,Honda,"119,157",-8.18,"1,092,247",-2.08,8.27,8.38
4,Nissan,"110,283",-13.29,"1,019,433",-5.83,7.65,7.82
5,Jeep,"83,764",14.11,"746,194",19.92,5.81,5.72
6,Subaru,"57,044",3.49,"503,418",5.13,3.96,3.86
7,Hyundai,"56,940",3.02,"492,791",-0.77,3.95,3.78
8,Kia,"51,503",-1.84,"452,042",-1.29,3.57,3.47
9,Ram,"56,447",9.21,"416,661",-0.49,3.92,3.20


In [89]:
raw_data = raw_data.loc[raw_data['Make'].isin(list(top_brands['Brand']))]

In [90]:
raw_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
11261,6500,2011,74520,Tacoma,WA,WAUSFAFL1BA115313,Audi,A44dr
11262,9791,2009,61473,Chandler,AZ,WAUAF48H79K009975,Audi,A42dr
11263,7495,2006,42065,Norfolk,VA,WAUAF78E46A113560,Audi,A44dr
11264,9100,2006,47500,Columbus,OH,WAUDF78E26A027502,Audi,A44dr
11265,7795,2008,73289,Delray Beach,FL,WAUAF78E98A138571,Audi,A44dr


In [91]:
#resetting indexes from 0 to nth order
raw_data.index = range(len(raw_data))

In [92]:
raw_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,6500,2011,74520,Tacoma,WA,WAUSFAFL1BA115313,Audi,A44dr
1,9791,2009,61473,Chandler,AZ,WAUAF48H79K009975,Audi,A42dr
2,7495,2006,42065,Norfolk,VA,WAUAF78E46A113560,Audi,A44dr
3,9100,2006,47500,Columbus,OH,WAUDF78E26A027502,Audi,A44dr
4,7795,2008,73289,Delray Beach,FL,WAUAF78E98A138571,Audi,A44dr


In [93]:
Counter(raw_data['Make'])

Counter({'Audi': 12618,
         'BMW': 32415,
         'Chevrolet': 102268,
         'Dodge': 34368,
         'Ford': 110432,
         'GMC': 29008,
         'Honda': 50193,
         'Hyundai': 35837,
         'Jeep': 40373,
         'Kia': 28636,
         'Lexus': 20641,
         'Mazda': 13365,
         'Mercedes-Benz': 26323,
         'Nissan': 66250,
         'Ram': 19808,
         'Subaru': 16428,
         'Toyota': 77786,
         'Volkswagen': 23249})

Now it seems that our data rows only contain the car brands correspondent to the YTD chart.
Since each car brand holds a different data size, I choose 1,000 sample size from each car brand.  

In [94]:
sample_data = pd.DataFrame({'Price': [],
                            'Year': [],
                            'Mileage': [],
                            'Make': []})

In [95]:
j = 0

for i in range(len(raw_data)-2):
    if raw_data['Make'][i] != raw_data['Make'][i+1]:
        sample_data = sample_data.append(raw_data[j:i].sample(n=1000))
        j = i
    if i == len(df)-3:
        sample_data = sample_data.append(raw_data[j:i].sample(n=1000))

In [96]:
#resetting indexes from 0 to nth order
sample_data.index = range(len(sample_data))

In [97]:
sample_data.head()

,City,Make,Mileage,Model,Price,State,Vin,Year
0,Upper Saddle River,Audi,20218.0,A63.0T,38777.0,NJ,WAUHGAFC2FN013203,2015.0
1,Sacramento,Audi,91919.0,A44dr,13488.0,CA,WAUAFAFL0CN011515,2012.0
2,San Antonio,Audi,14100.0,Q5quattro,42991.0,TX,WA1D7AFP6GA090325,2016.0
3,Indianapolis,Audi,84001.0,A8,39990.0,IN,WAUR4AFD0CN000788,2012.0
4,Addison,Audi,62654.0,Q5quattro,22495.0,TX,WA1WKAFP5CA056250,2012.0


In [98]:
Counter(sample_data['Make'])

Counter({'Audi': 1000,
         'BMW': 1000,
         'Chevrolet': 1000,
         'Dodge': 1000,
         'Ford': 1000,
         'GMC': 1000,
         'Honda': 1000,
         'Hyundai': 1000,
         'Jeep': 1000,
         'Kia': 1000,
         'Lexus': 1000,
         'Mazda': 1000,
         'Mercedes-Benz': 1000,
         'Nissan': 1000,
         'Ram': 1000,
         'Subaru': 1000,
         'Toyota': 1000,
         'Volkswagen': 1000})

In [99]:
 sample_data.to_csv("datasets/modified_car_brand.csv")